In [1]:
import sympy as smp
import time

def Christoffel(i,j,k):
    s = 0
    for a in range(n):
        s+=1/2*g_inv[a,i]*(smp.diff(g[a,j],coord[k])+smp.diff(g[a,k],coord[j])-smp.diff(g[j,k],coord[a]))
    return s

def Riemann(i,j,k,l):
    p=smp.diff(ch[i,j,l],coord[k])-(smp.diff(ch[i,j,k],coord[l]))
    s=0
    for o in range(n):
        s+=ch[i,o,k]*ch[o,j,l]-(ch[i,o,l]*ch[o,j,k])
    return smp.simplify(p+s)

def Ricci(i,j):
    s=0
    for o in range(n):
        s+=riem[o,i,o,j]
    return s

def Einstein(i,j):
    s=-1/2*g[i,j]*er+ricci[i,j]
    return s

In [2]:
coord = smp.symbols('t r theta phi', positive=True)
m, g2 = smp.symbols('m g', positive=True)

r1 = smp.symbols('r_1', cls=smp.Function)
r1=r1(coord[1])
valor_r1=(g2**2+coord[1]**2)**smp.Rational(1/2)

n = len(coord)

g= smp.Matrix([[0 for i in range(n)] for i in range(n)])
ch=  smp.MutableDenseNDimArray([[[0 for i in range(n)] for i in range(n)] for i in range(n)])
riem = smp.MutableDenseNDimArray([[[[0 for i in range(n)] for i in range(n)] for i in range(n)]for i in range(n)])
ricci = smp.Matrix([[0 for i in range(n)] for i in range(n)])
einstein = smp.Matrix([[0 for i in range(n)] for i in range(n)])

g[0,0]=-(1-(2*m*coord[1]**2)/(coord[1]**2+g2**2)**smp.Rational(3,2))
g[1,1]=1/(1-(2*m*coord[1]**2)/(coord[1]**2+g2**2)**smp.Rational(3,2))
g[2,2]=coord[1]**2
g[3,3]=coord[1]**2*smp.sin(coord[2])**2

g[0,0]=-(1-(2*m*coord[1]**2)/(r1**3))
g[1,1]=1/(1-(2*m*coord[1]**2)/r1**3)
g[2,2]=coord[1]**2
g[3,3]=coord[1]**2*smp.sin(coord[2])**2

g_inv = smp.simplify(g.inv())


gcovariante = smp.symbols(fr'g_alpha_beta')
gcontravariante = smp.symbols(fr'g^alpha^beta')

display(smp.Eq(gcovariante,g,evaluate=False))
display(smp.Eq(gcontravariante,g_inv,evaluate=False))

Eq(g_alpha_beta, Matrix([
[2*m*r**2/r_1(r)**3 - 1,                           0,    0,                  0],
[                     0, 1/(-2*m*r**2/r_1(r)**3 + 1),    0,                  0],
[                     0,                           0, r**2,                  0],
[                     0,                           0,    0, r**2*sin(theta)**2]]))

Eq(g^alpha^beta, Matrix([
[r_1(r)**3/(2*m*r**2 - r_1(r)**3),                       0,       0,                      0],
[                               0, -2*m*r**2/r_1(r)**3 + 1,       0,                      0],
[                               0,                       0, r**(-2),                      0],
[                               0,                       0,       0, 1/(r**2*sin(theta)**2)]]))

In [3]:
%%time
#Simbolos de Cristoffel
for a in range(n):
    for b in range(n):
        for c in range(n):
            ch[a,b,c]=(smp.simplify((Christoffel(a, b, c)).subs(smp.diff(r1,coord[1]),smp.diff(valor_r1,coord[1])))).subs(smp.sqrt(g2**2+coord[1]**2),r1)
            if ch[a, b, c] != 0:
                gamma = smp.Symbol(fr'\Gamma_{coord[b]}_{coord[c]}^{coord[a]}')
                display(smp.Eq(gamma,smp.simplify(ch[a, b, c]),evaluate=False))

Eq(\Gamma_t_r^t, -m*r*(3.0*r**2 - 2.0*r_1(r)**2)/((2*m*r**2 - r_1(r)**3)*r_1(r)**2))

Eq(\Gamma_r_t^t, -m*r*(3.0*r**2 - 2.0*r_1(r)**2)/((2*m*r**2 - r_1(r)**3)*r_1(r)**2))

Eq(\Gamma_t_t^r, -2*m*r*(3*r**2 - 2*r_1(r)**2)*(1.0*m*r**2 - 0.5*r_1(r)**3)/r_1(r)**8)

Eq(\Gamma_r_r^r, m*r*(6.0*r**2 - 4.0*r_1(r)**2)/((4.0*m*r**2 - 2.0*r_1(r)**3)*r_1(r)**2))

Eq(\Gamma_theta_theta^r, 2.0*m*r**3/r_1(r)**3 - 1.0*r)

...

Eq(\Gamma_r_phi^phi, 1.0/r)

Eq(\Gamma_theta_phi^phi, 1.0/tan(theta))

Eq(\Gamma_phi_r^phi, 1.0/r)

Eq(\Gamma_phi_theta^phi, 1.0/tan(theta))

CPU times: user 2.57 s, sys: 46 ms, total: 2.62 s
Wall time: 7.25 s


In [4]:
%%time
#Tensor de Riemann
for a in range(n):
    for b in range(n):
        for c in range(n):
            for d in range(n):
                riem[a,b,c,d]=((smp.simplify(Riemann(a, b, c,d))).subs(smp.diff(r1,coord[1]),smp.diff(valor_r1,coord[1]))).subs(smp.sqrt(g2**2+coord[1]**2),r1)
                if riem[a, b, c, d] != 0:
                    R = smp.Symbol(fr'R_{coord[b]}_{coord[c]}_{coord[d]}^{coord[a]}')
                    display(smp.Eq(R,riem[a, b, c,d],evaluate=False))

Eq(R_r_t_r^t, m*(-120.0*m*r**6 + 120.0*m*r**4*r_1(r)**2 - 16.0*m*r**2*r_1(r)**4 + 60.0*r**4*r_1(r)**3 - 60.0*r**2*r_1(r)**5 + 8.0*r_1(r)**7)/((16.0*m**2*r**4 - 16.0*m*r**2*r_1(r)**3 + 4.0*r_1(r)**6)*r_1(r)**4))

Eq(R_r_r_t^t, m*(120.0*m*r**6 - 120.0*m*r**4*r_1(r)**2 + 16.0*m*r**2*r_1(r)**4 - 60.0*r**4*r_1(r)**3 + 60.0*r**2*r_1(r)**5 - 8.0*r_1(r)**7)/((16.0*m**2*r**4 - 16.0*m*r**2*r_1(r)**3 + 4.0*r_1(r)**6)*r_1(r)**4))

Eq(R_theta_t_theta^t, 1.0*m*r**2*(-3.0*r**2 + 2.0*r_1(r)**2)/r_1(r)**5)

Eq(R_theta_theta_t^t, 0.5*m*r**2*(6.0*r**2 - 4.0*r_1(r)**2)/r_1(r)**5)

Eq(R_phi_t_phi^t, 1.0*m*r**2*(-3.0*r**2 + 2.0*r_1(r)**2)*sin(theta)**2/r_1(r)**5)

...

Eq(R_r_r_phi^phi, m*(-6.0*r**2 + 4.0*r_1(r)**2)/((4.0*m*r**2 - 2.0*r_1(r)**3)*r_1(r)**2))

Eq(R_r_phi_r^phi, m*(6.0*r**2 - 4.0*r_1(r)**2)/((4.0*m*r**2 - 2.0*r_1(r)**3)*r_1(r)**2))

Eq(R_theta_theta_phi^phi, -2.0*m*r**2/r_1(r)**3)

Eq(R_theta_phi_theta^phi, 2.0*m*r**2/r_1(r)**3)

CPU times: user 10.9 s, sys: 257 ms, total: 11.1 s
Wall time: 20.3 s


In [5]:
%%time
#Tensor de Ricci
for a in range(n):
    for b in range(n):
        ricci[a,b]=smp.simplify(Ricci(a,b))
        if ricci[a,b]!=0:
            Ri = smp.Symbol(fr'R_{coord[a]}_{coord[b]}')
            display(smp.Eq(Ri,smp.simplify(ricci[a,b]),evaluate=False))

Eq(R_t_t, 1.0*m*(30.0*m*r**6 - 42.0*m*r**4*r_1(r)**2 + 12.0*m*r**2*r_1(r)**4 - 15.0*r**4*r_1(r)**3 + 21.0*r**2*r_1(r)**5 - 6.0*r_1(r)**7)/r_1(r)**10)

Eq(R_r_r, m*(-480.0*r**4 + 672.0*r**2*r_1(r)**2 - 192.0*r_1(r)**4)/((64.0*m*r**2 - 32.0*r_1(r)**3)*r_1(r)**4))

Eq(R_theta_theta, 6.0*m*r**2*(-r**2 + r_1(r)**2)/r_1(r)**5)

Eq(R_phi_phi, 6.0*m*r**2*(-r**2 + r_1(r)**2)*sin(theta)**2/r_1(r)**5)

CPU times: user 907 ms, sys: 13.7 ms, total: 921 ms
Wall time: 946 ms


In [6]:
%%time
#Escalar de Ricci
er=0
for a in range(n):
    for b in range(n):
        er+=g_inv[a,b]*ricci[a,b]
ER=smp.symbols('R')
er = smp.ratsimp(er)
er = er.subs(r1,valor_r1)
er = smp.factor(er)
er = smp.simplify(er)
display(smp.Eq(ER,er,evaluate=False))

Eq(R, 24.0*g**2*m*(1.0*g - 0.5*r)*(1.0*g + 0.5*r)/(g**2 + r**2)**(7/2))

CPU times: user 235 ms, sys: 1.27 ms, total: 236 ms
Wall time: 241 ms


In [7]:
%%time
#Geodésicas
L=smp.symbols('lambda')
geod = [0 for i in range(n)]
for a in range(n):
    w1=0
    w2=0
    w1=smp.diff(coord[a],L,L,evaluate=False)
    for b in range(n):
        for c in range(n):
            w2+=smp.simplify(ch[a,c,b])*smp.diff(coord[c],L,evaluate=False)*smp.diff(coord[b],L,evaluate=False)
    display(coord[a])
    geod[a]=w1+w2
    display(smp.Eq(w1+w2,0,evaluate=False))

t

Eq(-2*m*r*(3.0*r**2 - 2.0*r_1(r)**2)*Derivative(r, lambda)*Derivative(t, lambda)/((2*m*r**2 - r_1(r)**3)*r_1(r)**2) + Derivative(t, (lambda, 2)), 0)

r

Eq(-2*m*r*(3*r**2 - 2*r_1(r)**2)*(1.0*m*r**2 - 0.5*r_1(r)**3)*Derivative(t, lambda)**2/r_1(r)**8 + m*r*(6.0*r**2 - 4.0*r_1(r)**2)*Derivative(r, lambda)**2/((4.0*m*r**2 - 2.0*r_1(r)**3)*r_1(r)**2) + r*(2.0*m*r**2 - 1.0*r_1(r)**3)*sin(theta)**2*Derivative(phi, lambda)**2/r_1(r)**3 + (2.0*m*r**3/r_1(r)**3 - 1.0*r)*Derivative(theta, lambda)**2 + Derivative(r, (lambda, 2)), 0)

theta

Eq(-0.5*sin(2*theta)*Derivative(phi, lambda)**2 + Derivative(theta, (lambda, 2)) + 2.0*Derivative(r, lambda)*Derivative(theta, lambda)/r, 0)

phi

Eq(Derivative(phi, (lambda, 2)) + 2.0*Derivative(phi, lambda)*Derivative(theta, lambda)/tan(theta) + 2.0*Derivative(phi, lambda)*Derivative(r, lambda)/r, 0)

CPU times: user 507 ms, sys: 5.58 ms, total: 513 ms
Wall time: 529 ms
